In [1]:
# this version go through both training and testing process
# need improvement
import os
import pickle
import numpy as np
import pandas as pd
from util import station_info, date_add_days, make_feat

In [2]:
data_path = './data/'
feature_path = './feature/'

In [3]:
model_dict,predictors = pickle.load(open(data_path+'lightgbm.model', 'rb+'))

In [4]:
# generate test set
def get_test_feat(date):
#     test_feat = make_feat(date_add_days(date[:10],1),replace=True)
    test_feat = make_feat(date_add_days(date[:10],1))
    test_feat = pd.concat([data_feat[:0],test_feat])
    return test_feat

In [5]:
# generate prediction for test set
def get_submission(test_feat,model_dict,decode):
    for label in ['PM2.5', 'PM10', 'O3']:
        pred = model_dict[label].predict(test_feat[predictors])
        test_feat[label] = decode[label](pred)
#     test_feat['test_id'] = test_feat['station_id'].apply(lambda x: x if len(x)<11 else x[:10])
    test_feat['test_id'] = test_feat['station_id']
    test_feat['test_id'] = list(map(lambda y: y+'_aq', test_feat['test_id']))
    test_feat['test_id'] = test_feat['test_id'] + '#' + test_feat['diff_of_hour'].astype(int).astype(str)
    submission = pd.read_csv(data_path + 'sample_submission.csv')
    submission = submission[['test_id']].merge(test_feat[['test_id','PM2.5','PM10','O3']],on='test_id',how='left')
    if (submission[['PM2.5','PM10','O3']]<0).sum().sum()>0:
        print('negative value!')
    return submission

In [6]:
start_date = '2018-03-24'
# range of days to compute feature
days = 446
data_feat_url = feature_path + 'data_feat_{}_{}days.hdf'.format(start_date,days)
data_feat = pd.read_hdf(data_feat_url, 'w')

In [7]:
def f1(x): return np.log(x+1)
def f2(x): return np.log(x+1)
def f3(x): return np.log(x+100)
def f4(x): return np.exp(x)-1
def f5(x): return np.exp(x)-1
def f6(x): return np.exp(x)-100
encode = {'PM2.5':f1,'PM10':f2,'O3':f3}
decode = {'PM2.5':f4,'PM10':f5,'O3':f6}

In [8]:
utc_date ='2017-04-30'

In [9]:
test_feat = get_test_feat(utc_date)

data key is：2017-05-01


In [10]:
test_feat_copy = test_feat.copy()

In [11]:
submission = get_submission(test_feat,model_dict, decode)

In [12]:
sub_url = r'./submission.csv'
submission.to_csv(sub_url,index=False,  float_format='%.4f')

In [13]:
def smape(actual, predicted):
    dividend= np.abs(np.array(actual) - np.array(predicted))
    denominator = np.array(actual) + np.array(predicted)
    
    return 2 * np.mean(np.divide(dividend, denominator, out=np.zeros_like(dividend), where=denominator!=0, casting='unsafe'))

In [14]:
# label = 'O3'
sma = 0
for label in ['PM2.5', 'PM10', 'O3']:
    sma += smape(test_feat_copy[label].fillna(0), submission[label])
sma/3

0.27050251376602563